In [1]:
#Libraries
import pandas as pd
import panel as pn
import numpy as np
import psycopg2
pn.extension('tabulator')

import hvplot.pandas
from holoviews import opts

In [2]:

# Connect to database
conn = psycopg2.connect(
    database = 'E-commerce_analysis',
    user = 'postgres',
    password= 'chan.chano123099',
    host='127.0.0.1',
    port =5000
)
conn.autocommit = True

cur = conn.cursor()

# Get data using queries
def get_data(query, conn):
    df = pd.read_sql_query(query, conn)
    return df
    
sales_profit_query = """
    select
        category,
        to_char(order_date, 'Month') as order_month,
        sum(amount) as monthly_revenue, sum(profit) as monthly_profit
    from order_details
    join list_of_orders On order_details.order_id = list_of_orders.order_id
    Group by category, order_month
    order by monthly_profit DESC
"""
city_sales_query = """
    select city,category,
    	count(category) as number_orders,
    	sum(amount) as Total
    from list_of_orders as A
    join order_details as B ON A.order_id = B.order_id
    group by  city, category
    order by number_orders DESC

"""

sales_product_query = """
    select category, subcategory, sum(amount) as Total_sales 
    from order_details
    group by category, subcategory
    order by total_sales DESC
"""
profit_sales_query = """
    select amount, profit, category, to_char(order_date, 'Month') as order_month
    from order_details as A
    join list_of_orders as B on A.order_id = B.order_id
"""

#All data from database
sales_trend = get_data(sales_profit_query, conn)
city_sales = get_data(city_sales_query , conn)
sales_product = get_data(sales_product_query, conn)
profit_sales = get_data(profit_sales_query, conn)

conn.close()

C:\Users\chano\AppData\Local\Temp\ipykernel_11184\3785893333.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [3]:
# Graph for Sales&Profit Trend per month

#Minimal date transformation
sales_trend['order_month'] = sales_trend['order_month'].str.strip()
sales_trend['order_month'] = pd.to_datetime(sales_trend['order_month'], format='%B').dt.month_name().sort_values().str.slice(stop=3)

#Mapping the order_month column for sorting
month_order = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
    'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

# Widgets for category
category = pn.widgets.RadioButtonGroup(name="Category", options=['Electronics', 'Clothing', 'Furniture'], button_type='success')


#Line for graph to become interactive
sales_trend_inter = sales_trend.interactive()

#Pipeline for sales_profit_trend to automatically update the graph
sales_trend_pipeline = sales_trend_inter[
    (sales_trend_inter.category == category )
].groupby(['order_month']).aggregate({'monthly_profit':'sum', 'monthly_revenue':'sum'}).sort_values(by = 'order_month', key=lambda x: x.map(month_order)).reset_index()

# Plotting the graph
trend_plot = sales_trend_pipeline.hvplot(x='order_month', 
                                         line_width = 2, 
                                         title= "Trends",
                                         height = 400, 
                                         width=800, 
                                         xlabel='Months', 
                                         grid=True, 
                                         color=['red','blue'], 
                                         cmap=['monthly_revenue','monthly_profit' ])
trend_plot.opts(legend_labels={'monthly_profit':"Profit", 'monthly_revenue':'Sales'}, legend_position='top_right')

In [5]:
# Graph for Sales per City

# Widgets for Top 10 and Bottom 10
select = pn.widgets.Select(name='Top or Bottom 10', options={'Bottom 10':True, 'Top 10':False})

#Line for graph to become interactive
city_sales_inter = city_sales.interactive()

#Pipeline for sales per city to automatically update the graph
city_sales_pipeline = city_sales_inter[
    (city_sales_inter.category == category)
].groupby(['city']).sum('total').sort_values(by='total', ascending=select).head(10)

#Plotting the graph
city_sales_plot = city_sales_pipeline.hvplot.bar(x='city', y='total').opts(title='Sales per City',
                                                                           ylabel='Sales', 
                                                                           xlabel='City', 
                                                                           bgcolor='lightgrey', 
                                                                           color='lightblue', 
                                                                           fontscale=1.2,
                                                                           xrotation=45, 
                                                                           height = 400, 
                                                                           width=800)
city_sales_plot

In [6]:
#Graphs for sales per product

#Line for graph to become interactive
sales_product_inter = sales_product.interactive()

#Pipeline
sales_product_pipeline = sales_product_inter[
   ( sales_product_inter.category == category)
].groupby(['subcategory']).sum('total_sales').sort_values('total_sales', ascending = False)

#Plot
sales_product_plot = sales_product_pipeline.hvplot.barh(x='subcategory', y='total_sales').opts(title='Sales per Product', 
                                                                                               xlabel='Products', 
                                                                                               ylabel='Sales',
                                                                                               height = 400, 
                                                                                               width=800, 
                                                                                               bgcolor = 'lightgrey', 
                                                                                               fontscale=1.2, 
                                                                                               color='lightblue')
sales_product_plot

In [7]:
#Graph for scatter Profit vs Sales

#For interactive graph
profit_sales_inter = profit_sales.interactive()

#Pipeline
profit_sales_pipeline = profit_sales_inter[
    (profit_sales_inter.category == category)
]

#Plot
profit_sale_plot = profit_sales_pipeline.hvplot.scatter(x='amount', y='profit', c='order_month').opts(title='Profit VS Sale', 
                                                                                                      width = 800, 
                                                                                                      height=400, 
                                                                                                      fontscale=1.2, 
                                                                                                      xlabel='Sales', 
                                                                                                      ylabel='Profit')
profit_sale_plot

In [19]:
#Using template to post in a localhost
template = pn.template.FastListTemplate(
    title = "E-commerce Analysis",
    main = [pn.Row(category, align="center"),
            pn.Row(pn.Column(trend_plot.panel(width=800), 
                             margin=(0,500,0,40)), 
                   pn.Column(profit_sale_plot.panel(width=800), 
                             margin=(0,20,0,0))),
            pn.Row(pn.Column(select, city_sales_plot.panel(width=800), 
                             margin=(0,0,0,40)),
                   sales_product_plot.panel(width=800))
                   ],
    accent_base_color = "#88d8b0",
    header_background = "#88d8b0",
                     
)

template.servable()

#To browse the dashboard. Type this in the terminal "panel serve Dashboard.ipynb"

FastListTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] TemplateActions()
    [browser_info] BrowserInfo(dark_mode=True, device_pixel_ratio=0.8999999761581421, language='en-US', timezone='Asia/Shanghai', timezone_offset=-480, webdriver=False)
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [1980185239504] Row(align='center')
        [0] RadioButtonGroup(button_type='success', name='Category', options=['Electronics', ...], value='Electronics')
    [1980189002480] Row
        [0] Column(margin=(0, 500, 0, 40))
            [0] ParamFunction(function, _pane=HoloViews, defer_load=False, width=800)
        [1] Column(margin=(0, 20, 0, 0))
            [0] ParamFunction(function, _pane=HoloViews, defer_load=False, width=800)
    [1980188403600] Row
        [0] Column(margin=(0, 0, 0, 40))
            [0] Select(name='Top or Bottom 10', options={'Bottom 10': True, ...}, value=True)
            [1] ParamFunction(function, _pane=HoloViews, defer_load=False, width=800)
        [1] ParamFunction(function, _pane=HoloViews, defer_load=False, width=800)